# Brown Dwarf Models

Modeling brown dwarfs is very similar to modeling thermal emission for exoplanets. The only difference is there is no stellar input!

In this tutorial you will learn:

1. How to turn that feature off 
2. Query a profile from the [Sonora Grid](https://zenodo.org/record/1309035#.Xo5GbZNKjGJ). Note, this is note necessary -- just convenient! 
3. Create a Brown Dwarf Spectrum 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import astropy.units as u

#picaso
import picaso.opacity_factory as opa_fa
from picaso import justdoit as jdi 
from picaso import justplotit as jpi
#plotting
from bokeh.io import output_notebook
from scipy.stats.stats import pearsonr  

output_notebook()
from bokeh.plotting import show,figure

Loading BokehJS ...

Start with the same inputs as before

In [23]:
wave_range = [2,3]
filename_db = '/home/ehsan/picaso/reference/opacities/opacities.db'
opa = jdi.opannection(wave_range=wave_range
                      ,filename_db=filename_db)
bd = jdi.inputs(calculation='browndwarf')

#wave_range = [3,5]
#opa = jdi.opannection(wave_range=wave_range)
#bd = jdi.inputs(calculation='browndwarf')

In [24]:
# Note here that we do not need to provide case.star, since there is none! 
bd.gravity(gravity=100 , gravity_unit=u.Unit('m/s**2'))

#this is the integration setup that was used to compute the Sonora grid 
#take a look at Spherical Integration Tutorial to get a look at what these parameters do
bd.phase_angle(0)


## Download and Query from Sonora Profile Grid 

Download the profile files that are located in the [profile.tar file](https://zenodo.org/record/1309035#.Xo5GbZNKjGJ)

Once you untar the file you can set the file path below. You do not need to unzip each profile. `picaso` will do that upon read in. `picaso` will find the nearest neighbor and attach it to your class. 

In [33]:
#point to where you untared your sonora profiles
sonora_profile_db = '/home/ehsan/picaso/document/data/Sonora/'
Teff = 2000 
#this function will grab the gravity you have input above and find the nearest neighbor with the 
#note sonora chemistry grid is on the same opacity grid as our opacities (1060). 
bd.sonora(sonora_profile_db, Teff)

In [34]:
#check what file was chosen
print(bd.inputs['atmosphere']['sonora_filename'])

t2000g100nc_m0.0.cmp.gz


In [35]:
#look at your input
bd.inputs['atmosphere']['profile'].head()

,pressure,temperature,e-,H2,H,H+,H-,H2-,H2+,H3+,...,SiO,MgH,OCS,Li,LiOH,LiH,LiCl,graphite,Al,CaH
0,0.000178,830.9,6.915421e-16,0.83681,9.897649e-11,4.500546e-38,3.066151e-26,2.710246e-36,4.500546e-38,4.500546e-38,...,2.568995e-20,8.834176e-29,8.119065e-10,1.011218e-12,2.907571e-10,3.745973e-15,9.077739e-10,4.500000e-38,8.392417e-56,8.392417e-56
1,0.000199,834.1,1.327390e-15,0.83681,1.896873e-10,4.500551e-38,9.332767e-26,1.166564e-35,4.500551e-38,4.500551e-38,...,1.380162e-19,5.693614e-28,7.996786e-10,1.468264e-12,2.801159e-10,5.367195e-15,1.081412e-09,4.500000e-38,7.808857e-56,7.808857e-56
2,0.000221,837.9,2.423602e-15,0.83681,3.459002e-10,4.500555e-38,2.651534e-25,4.632811e-35,4.500555e-38,4.500555e-38,...,6.558497e-19,3.239657e-27,7.883649e-10,2.078864e-12,2.725201e-10,7.577676e-15,1.271831e-09,4.500000e-38,7.161481e-56,7.161481e-56
3,0.000247,842.2,4.209275e-15,0.83681,6.001638e-10,4.500558e-38,7.031596e-25,1.697526e-34,4.500558e-38,4.500558e-38,...,2.756677e-18,1.627417e-26,7.779318e-10,2.870188e-12,2.677395e-10,1.054218e-14,1.476701e-09,4.500000e-38,6.473410e-56,6.473410e-56
4,0.000276,846.9,6.954026e-15,0.83681,9.908210e-10,4.500560e-38,1.740526e-24,5.738843e-34,4.500560e-38,4.500560e-38,...,1.024884e-17,7.217515e-26,7.683484e-10,3.864175e-12,2.656316e-10,1.445209e-14,1.692704e-09,4.500000e-38,5.767378e-56,5.767378e-56


## Run Spectrum

In [44]:
df = bd.spectrum(opa, full_output=True)

## Convert to $F_\nu$ Units and Regrid

`PICASO` outputs the raw flux as: 

$$ F_\lambda ( \frac{erc}{cm^2 * sec * cm}) $$

Typical fluxes shown in several Brown Dwarf papers are: 

$$ F_\nu ( \frac{erc}{cm^2 * sec * Hz}) $$

Below is a little example of how to convert units. 

**NOTE**: Some people like to plot out `Eddington Flux`, $H_\nu$. This gets confusing as the units appear to be erg/cm2/s/Hz but you will notice a factor of four difference: 

$$H_\nu = \frac{F_\nu}{4}$$

In [37]:
x,y = df['wavenumber'], df['thermal'] #units of erg/cm2/s/cm

xmicron = 1e4/x

flamy = y*1e-8 #per anstrom instead of per cm
sp = jdi.psyn.ArraySpectrum(xmicron, flamy, 
                            waveunits='um', 
                            fluxunits='FLAM')   
sp.convert("um")
sp.convert('Fnu') #erg/cm2/s/Hz

x = sp.wave #micron
y= sp.flux #erg/cm2/s/Hz
df['fluxnu'] = y 
x,y = jdi.mean_regrid(1e4/x, y, R=300) #wavenumber, erg/cm2/s/Hz
df['regridy'] =  y
df['regridx'] = x

## Compare with Sonora Grid

The corresponding spectra are also available at the same link above. `PICASO` doesn't provide query functions
for this. So if you want to compare, you will have to read in the files as is done below

In [42]:
son = pd.read_csv('sp_t900g100nc_m0.0',delim_whitespace=True, 
                 skiprows=3,header=None,names=['w','f'])
sonx, sony =  jdi.mean_regrid(1e4/son['w'], son['f'], newx=x)

In [43]:
show(jpi.spectrum([x]*2,[df['regridy'], sony], legend=['PICASO', 'Sonora']
                  ,plot_width=800,x_range=wave_range,y_axis_type='log'))
